In [1]:
with open('input.txt', 'r') as fl:
    for ln in fl:
        if ln.startswith('Register A:'):
            A = int(ln.split(' ')[-1])
        if ln.startswith('Register B:'):
            B = int(ln.split(' ')[-1])
        if ln.startswith('Register C:'):
            C = int(ln.split(' ')[-1])            
        if ln.startswith('Program:'):
            program = list(map(int, ln.strip().split(' ')[-1].split(',')))


In [2]:
A,B,C, program, len(program)

(22571680, 0, 0, [2, 4, 1, 3, 7, 5, 0, 3, 4, 3, 1, 5, 5, 5, 3, 0], 16)

In [3]:
def get_combo(operand, reg):
    if 0 <= operand <= 3:
        return operand
    elif operand == 4:
        return reg['A']
    elif operand == 5:
        return reg['B']
    elif operand == 6:
        return reg['C']
    else:
        raise ValueError("Cant Combo " + str(operand))

reg = {'A':A,'B':B,'C':C}
def run_program(program, reg, verbose=False):
    i = 0
    out = []
    while i < len(program):
        opcode = program[i]
        operand = program[i+1]
        if verbose:
            print(i, opcode, operand, reg, out)
        next_i = i + 2

        if opcode == 0: # adv/division
            reg['A'] = reg['A'] // (2 ** get_combo(operand, reg))
        elif opcode == 1: #blx
            reg['B'] = reg['B']^operand
        elif opcode == 2: #bst
            reg['B'] = get_combo(operand, reg) % 8
        elif opcode == 3: #jnz
            next_i = next_i if reg['A'] == 0 else operand
        elif opcode == 4: #bxc
            reg['B'] = reg['B'] ^ reg['C']
        elif opcode == 5: #out
            out.append(get_combo(operand, reg) % 8)
        elif opcode == 6: #bdv
            reg['B'] = reg['A'] // (2 ** get_combo(operand, reg))
        elif opcode == 7: #cdv
            reg['C'] = reg['A'] // (2 ** get_combo(operand, reg))

        i = next_i
    return out
run_program(program, reg, verbose=True)

0 2 4 {'A': 22571680, 'B': 0, 'C': 0} []
2 1 3 {'A': 22571680, 'B': 0, 'C': 0} []
4 7 5 {'A': 22571680, 'B': 3, 'C': 0} []
6 0 3 {'A': 22571680, 'B': 3, 'C': 2821460} []
8 4 3 {'A': 2821460, 'B': 3, 'C': 2821460} []
10 1 5 {'A': 2821460, 'B': 2821463, 'C': 2821460} []
12 5 5 {'A': 2821460, 'B': 2821458, 'C': 2821460} []
14 3 0 {'A': 2821460, 'B': 2821458, 'C': 2821460} [2]
0 2 4 {'A': 2821460, 'B': 2821458, 'C': 2821460} [2]
2 1 3 {'A': 2821460, 'B': 4, 'C': 2821460} [2]
4 7 5 {'A': 2821460, 'B': 7, 'C': 2821460} [2]
6 0 3 {'A': 2821460, 'B': 7, 'C': 22042} [2]
8 4 3 {'A': 352682, 'B': 7, 'C': 22042} [2]
10 1 5 {'A': 352682, 'B': 22045, 'C': 22042} [2]
12 5 5 {'A': 352682, 'B': 22040, 'C': 22042} [2]
14 3 0 {'A': 352682, 'B': 22040, 'C': 22042} [2, 0]
0 2 4 {'A': 352682, 'B': 22040, 'C': 22042} [2, 0]
2 1 3 {'A': 352682, 'B': 2, 'C': 22042} [2, 0]
4 7 5 {'A': 352682, 'B': 1, 'C': 22042} [2, 0]
6 0 3 {'A': 352682, 'B': 1, 'C': 176341} [2, 0]
8 4 3 {'A': 44085, 'B': 1, 'C': 176341} [2, 0

[2, 0, 1, 3, 4, 0, 2, 1, 7]

In [4]:
# part 2 ... 

# I noticed that, in general, A divided my 10 every "round" through the instructions
# and that earlier digits in A don't impact later etc. 

In [5]:
print('target:', program, len(program))

target: [2, 4, 1, 3, 7, 5, 0, 3, 4, 3, 1, 5, 5, 5, 3, 0] 16


In [6]:
# through experimentation, I found out that a number a bit bigger than
# 35000000000000 will get me to a 16 digit outcome
A0 = 35000000000000
_p = run_program(program, {'A':A0, 'B':B, 'C':C}, verbose=False)
_p, len(_p)

([6, 6, 6, 5, 6, 5, 3, 3, 5, 5, 4, 1, 4, 2, 1], 15)

In [7]:
"""
so I observed that roughly one order of magnitude in A leads to one digit in the result
being correct. once you get it, it doesn't change. 
however, there can be multiple solutions that are not contiguous, so we search by order
of magnitude, getting pickier by a single digit at a time until we get to the possible solutions.

i might have gotten lucky.
"""
As = [35000000000000]
results = []
ndigits = 1
mag = 10000000000000
for div in range(13):
    mag = mag // 10
    print(mag)
    found = False
    next_As = []
    for A0 in As:
        last_a = -10        
        for a in range(10000):
            out = run_program(program, {'A':A0+a*mag, 'B':B, 'C':C}, verbose=False)
            if tuple(out) == tuple(program):
                print('WE FINISHED!!', A0+a*mag)
                results.append(A0+a*mag)
            if len(out) == len(program):
                if tuple(out[-ndigits:]) == tuple(program[-ndigits:]):
                    found = True
                    if len(next_As) == 0:
                        next_As.append(A0+a*mag)
                    elif a > last_a + 1: # this is a new segment
                        next_As.append(A0+a*mag)
                    last_a = a
                    
    if not found:
        print('got as far as we could')
        break
    else:
        ndigits += 1
        As = list(set(next_As))

1000000000000
100000000000
10000000000
1000000000
100000000
10000000
1000000
100000
10000
1000
100
10
1
WE FINISHED!! 236581645540973
WE FINISHED!! 236581645541055
WE FINISHED!! 236580840234605
WE FINISHED!! 236580840234687
WE FINISHED!! 236580836040301
WE FINISHED!! 236580836040383


In [8]:
print(min(results))

236580836040301
